به نام خدا

پروژه‌ی سوم

# Naive Bayes Classifier
علیرضا توکلی
810197686

# هدف پروژه
در این پروژه قصد داریم با استفاده از الگوریتم بیز به تجزیه و تحلیل کامنت‌ها بپردازیم و مثبت یا منفی بودن نظرات را طبق متن آن‌ها تشخیص دهیم.

# توضیح کلی پروژه
در این پروژه می‌خواهیم با توجه به متن و عنوان کامنت‌ها، تشخیص دهیم که نویسنده‌ی آن، کالا را پیش‌نهاد کرده یا نه.
برای این کار از الگوریتم گفته شده استفاده خواهیم کرد. 

# دیتاهای لود شده
دو مجموعه داده در این راه حل لود می‌شود که هر دو مجموعه داده‌های دیجی‌کالا هستند و در فرمت سی‌اس‌وی می‌باشند. عنوان نظرات، خود نظر و مثبت بودن یا نبودن نظر در هر سطر داده‌ها آمده است. مجموعه داده‌های اول برای آموزش مدل ما به کار می‌آید و مجموعه‌ی دیگر برای تست عمل‌کرد مدل خواهد بود.

In [276]:
import pandas as pd
import numpy as np
from hazm import *

In [277]:
stopWords = False
smoothingComponent = 10
normalize = True

In [278]:
train = pd.read_csv('comment_train.csv')
test = pd.read_csv('comment_test.csv')

# فاز اول

## توضیح این فاز
در این فاز به مرتب کردن داده‌های متنی می‌پردازیم و روش‌های مختلف را امتحان می‌کنیم.

In [279]:
def dummy(x):
    return x

In [280]:
def normalizeDataset(data, function, stopwords = False, normalize = True):
    n, m = data.shape
    normalizer = Normalizer()
    for i in range(n):
        #print(data.iloc[i, 0])
        if normalize:
            data.iloc[i, 0] = normalizer.normalize(data.iloc[i, 0])
            data.iloc[i, 1] = normalizer.normalize(data.iloc[i, 1])
        data.iloc[i, 0] = list(map(function, \
                                   word_tokenize(data.iloc[i, 0])))
        if stopwords:
            for x in data.iloc[i, 0]:
                if x in stopwords_list():
                    data.iloc[i, 0].remove(x)
        data.iloc[i, 1] = list(map(function, \
                                   word_tokenize(data.iloc[i, 1])))
        if stopwords:
            for x in data.iloc[i, 1]:
                if x in stopwords_list():
                    data.iloc[i, 1].remove(x)

In [281]:
lemmatizer = Lemmatizer()
stemmer = Stemmer()
normalizeDataset(train, dummy, stopWords, normalize=normalize)
#normalizeDataset(train, lemmatizer.lemmatize, stopWords)
#normalizeDataset(train, stemmer.stem, stopWords)
train.head()

,title,comment,recommend
0,"[زیبا, اما, کم, دوام]","[با, وجود, سابقه, خوبی, که, از, برند, ایرانی, ...",not_recommended
1,"[بسیار, عالی]","[بسیار, عالی]",recommended
2,[سلام],"[من, الان, ۳, هفته, هست, استفاده, میکنم, برای,...",not_recommended
3,"[به, درد, نمیخورهههه]","[عمرش, کمه, تا, یه, هفته, بیشتر, نمیشه, استفاد...",not_recommended
4,"[کلمن, آب]","[فکر, کنین, کلمن, بخرین, با, ذوق, ., کلی, پولش...",not_recommended


In [282]:
normalizeDataset(test, dummy, stopWords, normalize=normalize)
#normalizeDataset(test, lemmatizer.lemmatize, stopWords)
#normalizeDataset(test, stemmer.stem, stopWords)
test.head()

,title,comment,recommend
0,"[وری, گود]","[تازه, خریدم, یه, مدت, کار, بکنه, مشخص, میشه, ...",recommended
1,"[زیاد, مناسب, نیست, رنگ, پس, میده, یه, وقتایی,...","[با, این, قیمت, گزینه‌های, بهتری, هم, میشه, گر...",not_recommended
2,"[پنکه, گوشی]","[خیلی, عالیه, ،, فقط, کاش, از, اون, سمتش, میشد...",recommended
3,"[دستگاه, خیلی, ضعیف]","[من, این, فیس, براس, چند, روز, یپش, به, دستم, ...",not_recommended
4,"[عالی, و, بیست]","[بنده, یه, هارد, اکسترنال, دارم, که, کابل, فاب...",recommended


## 1
تفاوت استیمینگ با لمیتیزشن:

در استیمینگ قسمت‌های اضافی ابتدا و انتهای کلمه، مثل ها، ام، اید و ... حذف می‌شوند.

اما در لمیتیزشن، کلمه به فرم ریشه‌ی واقعی‌اش در می‌آید. مثلا می‌روم تبدیل به رو می‌شود.

اما تاثیر هر روش:

| Algorithm | F1
| --- | --- |
| None | 0.883048620236531
| Lemmatizer | 0.8818897637795275
| Stemmer | 0.8748353096179183

نتایج نشان می‌دهد که اگر از هیچ یک از الگوریتم‌های هضم(به جز نرمالیزشن) استفاده نکنیم، نتیجه‌ی ما به مقدار خیلی کمی بهتر خواهد بود. دلیل آن نیز می‌تواند پیاده‌سازی بد توابع استیم و لمتازر باشد که مثلا در کلمه‌ی دوام، م را جزئی از کلمه نمی‌داند و آن را حذف می‌کند.

## نکته‌ی مهم در رابطه با استاپ ورد‌ها

استاپ ورد‌ها علاوه بر کند کردن قطعه کدمان، بسیار نتیجه را بد می‌کنند لذا در نتایج ما، استاپ ورد‌ها را اعمال نمی‌کنیم.

# فاز دوم

## توضیح این فاز
در این فاز به بررسی نحوه‌ی محاسبه‌ی احتمال هر دسته می‌پردازیم. از فرمول بیض استفاده می‌کنیم و برای رفع مشکل مطرح شده از اسموفینگ نیز بهره می‌بریم که توضیح بیش‌تر آن در ادامه آمده است.

## 2
احتمال این که با توجه به مشاهدات ما(که تعداد نمونه‌هایی با کلمه‌ی ایکس در این کامنت است) این نمونه از کلاس سی باشد، را

posterior

می‌گویند و توسط فرض اضافه‌ی الگوریتم نیو بیض که ویژگی‌ها را مستقل از هم می‌داند، محاسبه می‌شود. در مسئله‌ی ما ایکس هر کلمه‌ای و سی می‌تواند پیشنهاد شده یا نشده باشد. اگر این احتمال را به ازای همه‌ی کلمات داشته باشیم، کار تمام است.

احتمال این که بین همه‌ی نمونه‌هایی که عضو دسته‌ی سی هستند، ویژگی ایکس(تعداد کلمات آمده در کامنت) را داشته باشند،

likelihood

نام دارد که ما برای این که تخمینی از این احتمال داشته باشیم، روی تمامی داده‌های آموزشیمان که در کلاس سی هستند فور زده و تعداد آن‌هایی که ویژگی ایکس(این کلمه‌ی به‌خصوص) را دارند تقسیم بر تعداد کل می‌کنیم.

احتمال این که یک داده از دسته‌ی سی باشد را

Prior

می‌گوییم. به سادگی بر روی تمام نمونه‌های آموزشی حرکت کرده و تعداد اعضای هر کلاس را می‌شماریم. حال احتمال پریور یک کلاس برابر با تعداد آن کلاس تقسیم بر کل داده‌ها می‌شود. در مسئله‌ی ما احتمال این که یک فرد یک محصول را پیشنهاد بدهد یا ندهد است.

احتمال آخر که 

Evidence

نام دارد، احتمال این است که یک ویژگی در کل داده‌ها وجود داشته باشد. می‌توانیم برای تخمین آن فور بزنیم روی تمامی داده‌ها و آن‌هایی که این ویژگی را دارند بشماریم و تقسیم بر کل داده‌ها کنیم.  

In [283]:
words = set()
countDic = [{}, {}]
sumAll = [0, 0]
n, m = train.shape
for i in range(n):
    id = 0
    if train.iloc[i, 2] == 'recommended':
        id = 1
    for col in range(2):
        for word in train.iloc[i, col]:
            words.add(word)
            sumAll[id] += 1
            if word in countDic[id]:
                countDic[id][word] += 1
            else:
                countDic[id][word] = 1

## 3
زیرا در صورتی که کلمه‌ی ایکس‌یک فقط در کلاس سی‌یک آمده باشد، 

P(c|X) = P(c) * mul(P(x|c))

که به دلیل وجود کلمه‌ی ایکس یک در یکی از دسته‌ها داریم

P(x1|c1) != 0, P(x1|~c1) = 0

پس حتما

P(~c1|X) = 0 

خواهد شد. پس حتما دسته‌ی سی یک انتخاب می‌شود. زیرا دسته‌ی دیگر مقدار احتمالش صفر شده است.

## 4
در این روش ما متغیری به نام آلفا داریم که به آن اسموفینگ کامپوننت می‌گویند. این متغیر در صورت و مخرج کسر احتمالاتی ما به صورت جمع آمده و باعث می‌شود کسر ۰ نداشته باشیم. این روش پیاده‌سازی خاصی نیز ندارد و فقط کافی است در هنگام محاسبه کردن احتمال‌ها، مقدار از پیش تعیین شده را به صورت و مخرج کسرمان اضافه کنیم.

# فاز سوم

## توضیح این فاز
در این فاز به بررسی روش‌های نوشته شده و نحوه‌ی محاسبه‌ی دقت می‌پردازیم.

In [284]:
def testAndPrint(smoothing = True):
    n, m = test.shape
    
    correctDetectedRecommended = 0
    correctDetectedNotRecommended = 0
    allDetectedRecommended = 0
    totalRecommended = 0
    wrongcnt = 0
    print("5 Wrong samples: ")
    for i in range(n):
        #print(i, test.iloc[i, 0])
        prob = []
        for id in range(2):
            p = 1
            for key in test.iloc[i, 0]:
                if key in countDic[id]:
                    p *= (countDic[id][key] + smoothing * smoothingComponent)\
                            / (sumAll[id] + smoothing * smoothingComponent)
                else:
                    p *= smoothing * smoothingComponent / (sumAll[id] + smoothing * smoothingComponent)
            for key in test.iloc[i, 1]:
                if key in countDic[id]:
                    p *= (countDic[id][key] + smoothing * smoothingComponent)\
                            / (sumAll[id] + smoothing * smoothingComponent)
                else:
                    p *= smoothing * smoothingComponent / (sumAll[id] + smoothing * smoothingComponent)
            prob.append(p * sumAll[id] / (sumAll[0] + sumAll[1]))
        if prob[0] < prob[1]:
            allDetectedRecommended += 1
            if test.iloc[i, 2] == 'recommended':
                correctDetectedRecommended += 1
            elif wrongcnt < 5:
                print(test.iloc[i])
                wrongcnt += 1
        elif test.iloc[i, 2] == 'not_recommended':
            correctDetectedNotRecommended += 1
        elif test.iloc[i, 2] == 'recommended' and wrongcnt < 5:
            print(test.iloc[i])
            wrongcnt += 1
        if test.iloc[i, 2] == 'recommended':
            totalRecommended += 1
            
    accuracy = (correctDetectedNotRecommended + correctDetectedRecommended) / n
    precision = correctDetectedRecommended / allDetectedRecommended
    recall = correctDetectedRecommended / totalRecommended
    print("Accuracy is : ", accuracy)
    print("Precision is : ", precision)
    print("Recall is : ", recall)
    print("F1 is : ", 2 * precision * recall / (precision + recall))

## 5
هر کدام از این دو به تنهایی مفید نخواهد بود؛ زیرا در خیلی از موارد، ما مدل‌هایی با اندازه‌ی کلاس‌های برابر نداشته باشیم. مثلا ۹۹ درصد داده‌هایمان از یک کلاس باشند. آنگاه اگر کدمان همیشه کلاس ۹۹ درصدی را خروجی دهد، کار خاصی نکرده است. پس باید معیار دیگه‌ای را انتخاب کنیم.

برای مثال:

فرض کنید الگوریتم ما برای نصفی از حالات پیشنهاد شده و نشده درست پیش‌بینی می‌کند.
اگر محصول ما خوب باشد، پس کاربران زیادی پیش‌نهادش می‌کنند و فقط درصد کمی هستند که پیش‌نهادش نمی‌کنند. پس طبق فرمول، پرسیشن بالایی داریم و ریکال کمی.

همچنین اگر محصول ما محصول خوبی نباشد، طبق فرمول می‌رسیم به این که ریکال خوبی داریم و پرسیشن کمی. با این که الگوریتم ما کار مهمی نکرده و نصفی از داده‌های هر کلاس را اشتباه می‌گوید.

## 6
از این تابع استفاده می‌کنیم زیرا با مقادیر خیلی بزرگ مقابله می‌کند که اتفاقا این مقادیر بزرگ همان‌طور که در مثال بالا گفته شد، بسیار اتفاق می‌افتد. مثلا یکی از تابع‌ها نزدیک یک باشد و دیگری نزدیک ۰. اگر ما میانگین را اعمال کنیم به درصد ۵۰ می‌رسیم. اما در این فرمول ما هنوز هم اف‌یک کوچکی خواهیم داشت.

# 7
## a)

In [285]:
testAndPrint()

5 Wrong samples: 
title                                               [وری, گود]
comment      [تازه, خریدم, یه, مدت, کار, بکنه, مشخص, میشه, ...
recommend                                          recommended
Name: 0, dtype: object
title                                      [نقد, پس, از, خرید]
comment      [سلام, ،, راحت, شدم, از, کابل, شارژ, ،, توصیه,...
recommend                                          recommended
Name: 8, dtype: object
title                                                 [mi, ۴w]
comment      [کاور, مقاوم, و, قشنگیه, اما, متأسفانه, مدت, ز...
recommend                                      not_recommended
Name: 12, dtype: object
title                                           [گزینه, خوبیه]
comment      [سلام, ،, گزینه, خوبیه, من, یه, mg, ۲۵۴۰, خرید...
recommend                                          recommended
Name: 15, dtype: object
title                              [تستر, پرادا, کندی, فلورال]
comment      [من, این, ادو, تویلتو, خریدم, بر, خلاف, اینکه,...
recomm

## b)

In [269]:
testAndPrint()

Accuracy is :  0.88375
Precision is :  0.9299719887955182
Recall is :  0.83
F1 is :  0.8771466314398944


## c)

In [239]:
testAndPrint(False)

Accuracy is :  0.855
Precision is :  0.94375
Recall is :  0.755
F1 is :  0.8388888888888888


## d)

In [270]:
testAndPrint(False)

Accuracy is :  0.845
Precision is :  0.9509803921568627
Recall is :  0.7275
F1 is :  0.8243626062322947


| Algorithm | Accuracy | Precision | Recall | F1
| --- | --- | --- | --- | --- |
| a | 0.88875 | 0.9307479224376731 | 0.84 | 0.883048620236531
| b | 0.88375 | 0.9299719887955182 | 0.83 | 0.8771466314398944
| c | 0.855 | 0.94375 | 0.755 | 0.8388888888888888
| d | 0.845 | 0.9509803921568627 | 0.7275 | 0.8243626062322947

# 8
در حالت استفاده از هر دوی این حالات، نتیجه بسیار مطلوب هست همان‌طور که انتظار داشتیم. اگر هیچ‌یک را استفاده نکنیم نیز بدترین حالت خواهد بود. نکته‌ی مهم دیگر این است که تاثیر اسموتینگ مقداری بیش‌تر از نرمالایز کردن نوشته‌هاست که منطقی است زیرا تابع عالی‌ای برای این کار نوشته نشده است.

نکته‌ی مهم دیگر تفاوت پرسیشن و ریکال در نبودن سموفینگ است. که انتظار هم می‌رفت زیرا کلماتی که در وجود ندارند، همان‌طور که توضیح داده شد، باعث می‌شوند احتمال کلاسی صفر شود.

## 9
کلاس‌های غلط حدس زده شده:

title                                               [وری, گود]

comment      [تازه, خریدم, یه, مدت, کار, بکنه, مشخص, میشه, ...

recommend                                          recommended

title                                      [نقد, پس, از, خرید]

comment      [سلام, ،, راحت, شدم, از, کابل, شارژ, ،, توصیه,...

recommend                                          recommended

title                                                 [mi, 4w]

comment      [کاور, مقاوم, و, قشنگیه, اما, متأسفانه, مدت, ز...

recommend                                      not_recommended

title                                           [گزینه, خوبیه]

comment      [سلام, ،, گزینه, خوبیه, من, یه, mg, 2540, خرید...

recommend                                          recommended

title                              [تستر, پرادا, کندی, فلورال]

comment      [من, این, ادو, تویلتو, خریدم, \r, بر, خلاف, ای...

recommend                                          recommended

# راهکار‌های بهتر شدن اگوریتم و ادامه‌ی ۹
دلیل اشتباه در این تست‌ها به نظرم می‌تواند مشکل در تعداد کم بودن داده‌های آموزش باشد که الگوریتم با کلمه‌ی گود آشنا نباشد. می‌تواند بد بودن پیش‌پردازشمان باشد که ویرگول و بک‌اسلش آر را از این لیست‌ها حذف نکرده است یا مثلا نتوانسته به خوبی، خوبیه رو به ریشه‌ی خوب تبدیل کنه.